https://qiita.com/SaitoTsutomu/items/070ca9cb37c6b2b492f0

pandasとの連携で輸送問題を解く  


倉庫群から工場群へ部品を搬送したい。輸送費が最小となる計画を求めたい。
- 倉庫群から工場群への輸送量を決めたい → 変数
- 輸送コストを最小化したい → 目的関数
- 各倉庫からの搬出は、供給可能量以下 → 制約
- 各工場への搬入は、需要量以上 → 制約


In [1]:
import numpy as np
import pandas as pd
import pulp
from itertools import product

In [2]:
np.random.seed(1)

In [3]:
nw, nf = 3, 4

In [4]:
pr = list(product(range(nw), range(nf)))
pr

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3)]

In [5]:
供給 = np.random.randint(30, 50, nw)
需要 = np.random.randint(20, 40, nf)
輸送費 = np.random.randint(10, 20, (nw,nf))

In [6]:
供給

array([35, 41, 42])

In [7]:
需要

array([28, 29, 31, 25])

In [8]:
輸送費

array([[10, 10, 11, 17],
       [16, 19, 12, 14],
       [15, 12, 14, 12]])

In [9]:
a = pd.DataFrame([(i,j) for i, j in pr], columns=['倉庫', '工場'])
a

,倉庫,工場
0,0,0
1,0,1
2,0,2
3,0,3
4,1,0
5,1,1
6,1,2
7,1,3
8,2,0
9,2,1


In [10]:
# problem
p = pulp.LpProblem()

In [11]:
a['輸送費'] = 輸送費.flatten()
a['Var'] = [pulp.LpVariable('v%d'%i, lowBound=0) for i in a.index]
a

,倉庫,工場,輸送費,Var
0,0,0,10,v0
1,0,1,10,v1
2,0,2,11,v2
3,0,3,17,v3
4,1,0,16,v4
5,1,1,19,v5
6,1,2,12,v6
7,1,3,14,v7
8,2,0,15,v8
9,2,1,12,v9


In [12]:
# tareget
p += pulp.lpDot(a.輸送費, a.Var)

In [13]:
# subjects
for k, v in a.groupby('倉庫'):
    p += pulp.lpSum(v.Var) <= 供給[k]
for k, v in a.groupby('工場'):
    p += pulp.lpSum(v.Var) >= 需要[k]
p

NoName:
MINIMIZE
10*v0 + 10*v1 + 14*v10 + 12*v11 + 11*v2 + 17*v3 + 16*v4 + 19*v5 + 12*v6 + 14*v7 + 15*v8 + 12*v9 + 0
SUBJECT TO
_C1: v0 + v1 + v2 + v3 <= 35

_C2: v4 + v5 + v6 + v7 <= 41

_C3: v10 + v11 + v8 + v9 <= 42

_C4: v0 + v4 + v8 >= 28

_C5: v1 + v5 + v9 >= 29

_C6: v10 + v2 + v6 >= 31

_C7: v11 + v3 + v7 >= 25

VARIABLES
v0 Continuous
v1 Continuous
v10 Continuous
v11 Continuous
v2 Continuous
v3 Continuous
v4 Continuous
v5 Continuous
v6 Continuous
v7 Continuous
v8 Continuous
v9 Continuous

In [14]:
# solve
p.solve()

1

In [15]:
a['Val'] = a.Var.apply(pulp.value)
a

,倉庫,工場,輸送費,Var,Val
0,0,0,10,v0,28.0
1,0,1,10,v1,7.0
2,0,2,11,v2,0.0
3,0,3,17,v3,0.0
4,1,0,16,v4,0.0
5,1,1,19,v5,0.0
6,1,2,12,v6,31.0
7,1,3,14,v7,5.0
8,2,0,15,v8,0.0
9,2,1,12,v9,22.0


In [16]:
a[a.Val > 0]

,倉庫,工場,輸送費,Var,Val
0,0,0,10,v0,28.0
1,0,1,10,v1,7.0
6,1,2,12,v6,31.0
7,1,3,14,v7,5.0
9,2,1,12,v9,22.0
11,2,3,12,v11,20.0
